# Classifier Evaluation

In [ ]:
from sklearn.metrics import confusion_matrix, plot_confusion_matrix,\
    precision_score, recall_score, accuracy_score, f1_score, log_loss,\
    roc_curve, roc_auc_score
import numpy as np
from sklearn.utils import resample
from sklearn.datasets import load_breast_cancer, load_iris, make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from matplotlib import pyplot as plt

## Agenda

SWBAT:

- define cross-entropy loss;
- define a confusion matrix;
- describe different classification metrics such as accuracy, recall, and precision;
- characterize AUC-ROC as a classifier metric.

## Dummy Classification Data

In [ ]:
X, y = make_classification(flip_y=0.1,
                           random_state=42)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   random_state=42)

In [ ]:
logit_model = LogisticRegression(random_state=42)

logit_model.fit(X_train, y_train)

In [ ]:
y_pred = logit_model.predict_proba(X_test)

### Predictions

***`.predict()` vs. `.predict_proba()`***

In [ ]:
logit_model.predict_proba(X_test)[:5]

In [ ]:
logit_model.predict(X_test)[:5]

## Loss Functions

One natural way of measuring the quality of a classifier is just to look at the loss function, which will often be log loss or, in the case of multiple classes, **cross-entropy loss**.

In [ ]:
log_loss(y_test, y_pred)

### Log Loss by Hand

Log loss is generally calculated as an average per data point, and is computed as follows:

$L(y, \hat{y}) = -\frac{1}{N}\sum^N_{i=1}[y_i\ln(\hat{y_i}) + (1-y_i)\ln(1-\hat{y_i})]$,

where $y$ is the vector of true values and $\hat{y}$ is the vector of probabilities that the point in question has a correct label of 1.

- Suppose, for a given data point, that the correct prediction of the label is **0**. In that case, the contribution from that point to the sum in the loss function defined above will be $-\ln(1-\hat{y_i})$. So, the closer the prediction for that point is to 0, the closer the contribution to the sum will be to $-\ln(1)=0$. But as the prediction gets closer to 1, the closer the contribution will be to $-\ln(0)=\infty$.

- Suppose, on the other hand, that the correct prediction is **1**. In that case, the contribution from that point to the sum in the loss function defined above will be $-\ln(\hat{y_i})$. So, the closer the prediction for that point is to 1, the closer the contribution to the sum will be to $-\ln(1)=0$. But as the prediction gets closer to 0, the closer the contribution will be to $-\ln(0)=\infty$.

In [ ]:
compare = list(zip(y_test, y_pred))

In [ ]:
compare[:5]

In [ ]:
calc = [-(yi * np.log(yi_hat[1]) + (1 - yi) * np.log(yi_hat[0])) for (yi, yi_hat) in compare]
calc[:5]

In [ ]:
np.mean(calc)

This is not so straighforwardly interpretable, however, and very often classifier metrics will instead make appeal to the *confusion matrix* associated with a model.

## Confusion Matrix

For classification problems, the target is a categorical variable. This means that we can simply count the number of times that our model predicts the correct category and the number of times that it predicts something else.

We can visualize this by means of a **confusion matrix**, which displays counts of correct and incorrect predictions. We'll explore this below. There are [many ways](https://docs.google.com/document/d/12BkMOXt5-iMzw_y_zFLZNkPlgAy1rj1t9gJ6LXi2bqQ/edit?usp=sharing) of evaluating a classification model, but most derive from the confusion matrix.

## Lottery Number Prediction

Suppose I want to train a model to predict whether a string of six numbers (a "ticket") would win the lottery or not. What sort of data might I use?

### Scenario 1

I gather all the winning tickets from the last 10000 days or so. So I have one column for the tickets themselves, and a Boolean column indicating whether the ticket was a winner or not.

Now if all the tickets on which my model trains are *winning* tickets, then it would predict every ticket to win! Suppose I test the model on a set of 1000 tickets, and suppose that there is exactly one winning ticket among those 1000. My model will always predict the ticket to win. Let's think about what the confusion matrix will look like.

In [ ]:
# Setting up the true values

y_true = np.zeros(1000)
y_true[500] = 1

y_true

In [ ]:
# Setting up the predictions

y_pred = np.ones(1000)

y_pred

In [ ]:
# Defining the confusion matrix

cm_1 = confusion_matrix(y_true, y_pred)

The confusion matrix should tell us that we have 999 false positives (999 losing tickets predicted to win) and 1 true positive (1 winning ticket predicted to win):

In [ ]:
cm_1

Notice the way that sklearn displays its confusion matrix: The rows are \['actually false', 'actually true'\]; the columns are \['predicted false', 'predicted true'\].

So it displays:

$\begin{bmatrix}
TN & FP \\
FN & TP
\end{bmatrix}$

In [ ]:
tn = cm_1[0, 0]
fp = cm_1[0, 1]
fn = cm_1[1, 0]
tp = cm_1[1, 1]

Let's see if we can calculate some of our metrics for this matrix.

**Accuracy** = $\frac{TP + TN}{TP + TN + FP + FN}$

In words: How often did my model get the right answer?

In [ ]:
# Accuracy

print(f'Accuracy: ({tp} + {tn}) / ({tp} + {tn} + {fp} + {fn}) = 1/1000 = {1/1000}')

**Recall** = $\frac{TP}{TP + FN}$

In words: How often did my model correctly predict winning tickets?

In [ ]:
# Recall

print(f'Recall: {tp} / ({tp} + {fn}) = 1/1 = {1}')

**Precision** = $\frac{TP}{TP + FP}$

In words: How often was my model's prediction of 'winner' correct?

In [ ]:
# Precision

print(f'Precision: {tp} / ({tp} + {fp}) = 1 / 1000 = {1/1000}')

**F-1 Score** = $\frac{2PrRc}{Pr + Rc}$ = $\frac{2TP}{2TP + FP + FN}$

In [ ]:
# F-1 Score

print(f'F-1 Score: (2)({tp}) / ((2)({tp}) + {fp} + {fn}) = 2 / 1001 = {round(2/1001, 3)}')

### Scenario 2

Well, my recall was good, but everything else I measured was terrible! Can I do better?

This time I'll train my model in a much different way. I'll give it the tickets of 10000 people who played the lottery yesterday. Suppose that there are one winning ticket and 9999 losers. Now I test the model on the same 1000 tickets from before in Scenario 1.

This time my model will almost always predict a ticket to lose. Suppose that, in the 1000 predictions, it makes only one prediction of a winner, and suppose that this prediction is wrong.

#### Exercise: Set up the confusion matrix and compute our metrics for this new scenario.

### Lessons

The last classifier had a really high accuracy, but everything else was terrible.

It occurs to me now that if I'm really going to get a good model running, I'm going to have to have good numbers of **both winning and losing  tickets**. The first model didn't see enough losers, and the second model didn't see enough winners.

Of course, I could just be more careful about my data collection, but there are often easier fixes. One of the most effective strategies is **oversampling the minority class**. That is, I give myself more data points than I really have. I could achieve this either by [bootstrapping](https://scikit-learn.org/stable/modules/generated/sklearn.utils.resample.html) or by generating some data that is fake but close to actual data. The latter is the idea behind [SMOTE](https://imbalanced-learn.org/stable/over_sampling.html).

## Breast Cancer Prediction

Let's see what results we get on scikit-learn's breast cancer dataset:

### Loading the Data

In [ ]:
preds, target = load_breast_cancer(return_X_y=True)

One way to measure the relative size of our classes:

In [ ]:
sum(target) / len(target)

Let's assume that this is close enough to "balanced"!

### Splitting and Scaling:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(preds, target,
                                                   random_state=42)

In [ ]:
ss = StandardScaler()

ss.fit(X_train)

X_train_sc = ss.transform(X_train)
X_test_sc = ss.transform(X_test)

### Model Fitting

In [ ]:
logreg = LogisticRegression(solver='lbfgs', max_iter=10000,
                           random_state=42)

logreg.fit(X_train_sc, y_train)

### Confusion Matrix

In [ ]:
confusion_matrix(y_test, logreg.predict(X_test_sc))

### Interpretation

#### Exercise: What are the precision, recall, and accuracy of our model?

## Multiple Classes

We can understand these metrics of recall, precision, and the rest even if there are more than two classes in our classification problem.

In [ ]:
flowers = load_iris()

In [ ]:
print(flowers.DESCR)

In [ ]:
dims_train, dims_test, spec_train, spec_test = train_test_split(flowers.data,
                                                                flowers.target,
                                                                test_size=0.5,
                                                               random_state=42)

In [ ]:
spec_train[:5]

In [ ]:
ss_f = StandardScaler()

ss_f.fit(dims_train)

dims_train_sc = ss_f.transform(dims_train)
dims_test_sc = ss_f.transform(dims_test)

In [ ]:
logreg_f = LogisticRegression(multi_class='multinomial',
                             C=0.01, random_state=42)

logreg_f.fit(dims_train_sc, spec_train)

In [ ]:
plot_confusion_matrix(estimator=logreg_f,
                      X=dims_test_sc,
                      y_true=spec_test,
                     display_labels=[
                         'setosa',
                         'versicolor',
                         'virginica'
                            ]);

In [ ]:
accuracy_score(spec_test,
              logreg_f.predict(dims_test_sc))

In [ ]:
(29 + 15 + 22) / (spec_test.shape[0])

In [ ]:
precision_score(spec_test,
                logreg_f.predict(dims_test_sc),
               average=None)

In [ ]:
print(29/29, 15/16, 22/30)

In [ ]:
recall_score(spec_test,
            logreg_f.predict(dims_test_sc),
            average=None)

In [ ]:
print(29/29, 15/23, 22/23)

For multi-class precision, the relevant denominator is a **column**; for multi-class recall, the relevant denominator is a **row**.

## Which Metric Should I Care About?

Well, it depends.

### General Lessons

First, let's make some general observations about the metrics we've so far defined.

Accuracy:
- Pro: Takes into account both false positives and false negatives.
- Con: Can be misleadingly high when there is a significant class imbalance. (A lottery-ticket predictor that *always* predicts a loser will be highly accurate.)

Recall:
- Pro: Highly sensitive to false negatives.
- Con: No sensitivity to false positives.

Precision:
- Pro: Highly sensitive to false positives.
- Con: No sensitivity to false negatives.

F-1 Score:
- Harmonic mean of recall and precision.

### Cost

The nature of your business problem will help you determine which metric matters.

Sometimes false positives are much worse than false negatives: Arguably, a model that compares a sample of crime-scene DNA with the DNA in a city's database of its citizens presents one such case. Here a false positive would mean falsely identifying someone as having been present at a crime scene, whereas a false negative would mean only that we fail to identify someone who really was present at the crime scene as such.

On the other hand, consider a model that inputs X-ray images and predicts the presence of cancer. Here false negatives are surely worse than false positives: A false positive means only that someone without cancer is misdiagnosed as having it, while a false negative means that someone with cancer is misdiagnosed as *not* having it.

### Strategies

There are a couple of strategies for trying to quantify these important differences in value between false positives and false negatives.

#### Strategy 1: Use a cost matrix.

One might assign different weights to the costs associated with false positives and false negatives. (We'll standardly assume that the costs associated with *true* positives and negatives are negligible.)

**Example**. Suppose we are in the DNA prediction scenario above. Then we might construct the following cost matrix:

In [ ]:
cost = np.array([[0, 10], [3, 0]])
cost

This cost matrix will allow us to compare models if we have access to those models' rates of false positives and false negatives, i.e. if we have access to the models' confusion matrices!

**Problem**. Given the cost matrix above and the confusion matrices below, which model should we go with?

In [ ]:
conf1, conf2 = np.array([[100, 10], [30, 300]]), np.array([[120, 20], [0, 300]])

print(conf1, 2*'\n', conf2)

#### Strategy 2: Adjust the thresholds for false positives and false negatives.

Let's turn to this now!

## The Area Under the Curve of the Receiver Operating Characteristic

### Data

First let's generate some data using sklearn's make_classification tool:

In [ ]:
X, y = make_classification(n_samples=10000, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
ss = StandardScaler()

X_train_sc = ss.fit_transform(X_train)
X_test_sc = ss.transform(X_test)

### Model

Now we'll fit a LogisticRegression object:

In [ ]:
logreg = LogisticRegression(solver='liblinear')

logreg.fit(X_train_sc, y_train)

### Confusion Matrix

In [ ]:
cm = confusion_matrix(y_test, logreg.predict(X_test_sc))

In [ ]:
cm

Defining true/false positives/negatives:

In [ ]:
tp, tn, fp, fn = cm[1][1], cm[0][0], cm[0][1], cm[1][0]

### AUC-ROC

The Receiver Operating Characteristic curve plots the true-positive rate vs. the false-positive rate. Let's define these now:

In [ ]:
tpr = tp / (tp + fn)
fpr = fp / (fp + tn)

#### Thresholds

Wait. How does this make sense? Doesn't a classifier just have a certain number of true positives, false positives, and all the rest? And so wouldn't a "plot" of these rates just be a single point on a graph?

Consider a prediction for a particular data point. The features have particular values that lead the model to predict 0 or 1, one way or the other. But the model doesn't merely spit out 0's and 1's: As we just saw, there is a *calculation* done here. Let's look again at the predicted probabilities of class membership for a particular point:

In [ ]:
logreg.predict_proba(X_test_sc)[0]

In [ ]:
logreg.predict(X_test_sc)[0]

Now the default behavior is simply to take the larger of these values as the "real" prediction. Since $0.84 > 0.16$, we'll understand the model to be predicting this point to belong to class "1" (or the positive class). An equivalent way of understanding the default behavior is that we:

- round the predicted numbers up to 1 if they are at least as large as 0.5; and
- round them down to 0 if they are less than 0.5.

Since the probabilities must sum to 1, there will never be any problem with this algorithm.

But we don't have to do things this way. Suppose we're building a model that predicts the presence of prostate cancer from X-ray scans of prostates. And suppose we get a pair of probabilities for some particular scan that look like this:

- pred_neg: 0.52, pred_pos: 0.48

Because false negatives (cancerous prostates mislabeled) are *much* more costly than false positives (non-cancerous prostates mislabeled), we may well want to **adjust our threshold** of classification. We might want to have our model predict "positive" if the corresponding probability is, say, as low as 0.4, or maybe even as low as 0.1. (Speaking for myself, if there was even a 10% chance that my prostate was cancerous, I think I'd probably want to know about it.)

Clearly, the true- and false-positive rates will change if we make this adjustment to the threshold. In fact, in the present case that was the whole point of making the adjustment: We want to minimize our false negatives.

So this is how the plot of these rates takes shape.

Let's build a function that will take in our data, together with a threshold setting, and return the corresponding true- and false-positive rates.

In [ ]:
def classify_rates(X_train, X_test, y_train, y_test, thresh):
    logreg = LogisticRegression(solver='liblinear')
    logreg.fit(X_train, y_train)
    y_hat_probs = logreg.predict_proba(X_test)
    y_hat = []
    for val in y_hat_probs:                                 # Each element in y_hat_probs is an array.
        if val[0] < thresh:                                 # We'll set our own threshold for classifying
            y_hat.append(1)                                 # a test point as positive! The lower my threshold,
        else:                                               # the fewer predicted positives I'll have. For the
            y_hat.append(0)                                 # cancer example, I'd want to set a *high* threshold.
    cm = confusion_matrix(y_test, y_hat)
    tp, tn, fp, fn = cm[1][1], cm[0][0], cm[0][1], cm[1][0]
    tpr = tp / (tp + fn)
    fpr = fp / (fp + tn)
    return tpr, fpr, f'tpr:{round(tpr, 3)}, fpr:{round(fpr, 3)}'

True- and false-positive rates for various thresholds:

In [ ]:
for x in np.linspace(0, 1, 11):
    print(f'Rates at threshold = {round(x, 1)}: ' + classify_rates(X_train_sc, X_test_sc, y_train, y_test, x)[2])

As my threshold goes up, I'll have more positive predictions, which means I'll have both more true positives and more false positives.

Note:

- I can artificially increase my true-positive rate to 1 by setting my threshold to 1, but at that point my false-positive rate is also 1! I'll have no true negatives and no false negatives. This will arise naturally if my training data has **very few (actual) negatives**. This was the problem in Lottery Scenario 1.
- I can artificially reduce my false-positive rate to 0 by setting my threshold to 0, but at  that point my true-positive rate is also 0! I'll have no true positives and no false positives. This will arise naturally if my training data has **very few (actual)  positives**. This was the problem in Lottery Scenario 2.

#### Area Under the Curve

The ROC curve will be a plot of tpr (on the y-axis) vs. fpr (on the x-axis). There will always be a point at (0, 0) and another at (1, 1). The question is what happens in the middle. Since we want our y-values to be as high as possible for any particular x-value, a natural metric is to calculate the **area under the curve**. The larger the area, the better the classifier. The maximum possible area is the area of the whole box between 0 and 1 on both axes, so that's a **maximum area of 1**.

What's the minimum? Well that depends on the ratios of (actual) positive and negatives in my data, in much the way that a baseline accuracy score does.

Remember: If my test data comprises 90% positives and only 10% negatives, then a simple classifier that always predicts "positive" will be 90% accurate! And so that would be the baseline level for a classifier on that data.

If we have equal numbers of positives and negatives, then we can set an **abolute minimum area of 0.5**. That's the "curve" we'd get by plotting a straight diagonal line from (0, 0) to (1, 1).

Why? The area under the curve really represents the test's ability to **discriminate** positives from negatives. Suppose I randomly took several pairs of points, one positive and one negative, and checked my test's predictions. The area under the curve represents a threshold-independent measure of how often my test would get the two predictions correct.

#### Plotting the Curve

Let's plot our own ROC curve. We'll create an array of different thresholds and use our `classify_rates()` function to get the true- and false-positive rates for each threshold.

One way of choosing a threshold **independently of business concerns** is to select the point on the curve that is furthest from (1, 0), the "worse-case" point where our true-positive rate is 0 and our false-positive rate is 1. So let's find that point as well:

In [ ]:
tprs = []
fprs = []
diffs = []
for x in np.linspace(0, 1, 101):
    fprs.append(classify_rates(X_train_sc, X_test_sc, y_train, y_test, x)[1])
    tprs.append(classify_rates(X_train_sc, X_test_sc, y_train, y_test, x)[0])
    diffs.append(np.sqrt(tprs[-1]**2 + (1-fprs[-1])**2))
    
max_dist = diffs.index(np.max(diffs))
print(f"""With a threshold of {(max_dist - 1) / 100}: \n"""
      f"""\tYou\'ll have a True Positive Rate of {round(tprs[max_dist], 3)} \n"""
      f"""\tand a False Positive Rate of {round(fprs[max_dist], 3)}""")

fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(fprs[:max_dist], tprs[:max_dist], 'r.')
ax.plot(fprs[max_dist], tprs[max_dist], 'ko', ms=10)
ax.plot(fprs[max_dist + 1:], tprs[max_dist + 1:], 'r.')
ax.plot(fprs, fprs, '.');

Scikit-Learn's `roc_auc_score()` function will compute the area under the curve for us:

In [ ]:
round(roc_auc_score(y_test, logreg.predict(X_test_sc)), 4)

Let's compare our curve with scikit-learn's:

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, logreg.predict(X_test_sc))

In [ ]:
fig, ax = plt.subplots()
ax.plot(fpr, tpr);

Scikit-Learn only shows us the optimal threshold, but it appears to be very similar to ours.